# databases and `aws`

what if you want to do something with data beyond just dumping it into an `s3` bucket?

`aws` has [several database options](https://aws.amazon.com/free/databases-free-tier/), but we're going to talk about two in particular:

1. `rds` (Relational Database Service): several common `sql rdbms` running on managed `ec2` servers
2. `dynamodb`: an `aws`-specific `nosql` database service

there are also several database options that `aws` doesn't support natively, and of those, we will (time permitting) focus on one: `neo4j`

## understanding `sql`, `rdbms`, and `nosql`

from about 2010 to 2016 or so every blogger on the internet wrote an article explaining the difference between `nosql` and `sql` (or specifically: `rdbms`). I won't even pretend to be able to do a better job than them. Here are a few worth reading:

1. [the wikipedia `nosql` entry](https://en.wikipedia.org/wiki/NoSQL)
2. [a good blog post motivating why one might choose `nosql`](https://www.upwork.com/hiring/data/sql-vs-nosql-databases-whats-the-difference/)
3. [a random stack exchange link that 57 people on the internet like](https://dba.stackexchange.com/questions/5/what-are-the-differences-between-nosql-and-a-traditional-rdbms)
4. [another good blog post focusing on the differences between the two](http://www.thegeekstuff.com/2014/01/sql-vs-nosql-db/?utm_source=tuicool)

what follows is about as high-level an introduction to these topics as I can give. If you'd like to know more, you have a lifetime to ~\*~LeArN~\*~!!

### the basics: `sql`

`sql` (Structured Query Language) has existed since the 1970s, been an ANSI ([American National Standards Insitute](https://en.wikipedia.org/wiki/American_National_Standards_Institute)) standard since 1986, and an ISO ([International Organization for Standardization](https://en.wikipedia.org/wiki/International_Organization_for_Standardization)) standard since 1987.

show of hands: who here is older than ANSI-standard `sql`?

you've learned `sql` in 510 and almost certainly used it in data applications. it's everywhere. 

people create perverse monstrocities within real programming language to allow people to write queries in `sql` instead (I'm looking at you, [sqldf](https://cran.r-project.org/web/packages/sqldf/index.html) and [pandassql](https://github.com/yhat/pandasql). you should be ashamed of yourselves)

you cannot escape it.

which is a good thing, I suppose -- having one universally accepted standard for how you interact with large, normalized, relational data means that you only need to learn it once, and large user communities mean that the standards will be both rigorous and evolving.

### the basics: `rdbms`

it is somewhat common for people to *say* `sql` but actually *talk* about `rdbm`s ([Relational DataBase Management Systems](https://en.wikipedia.org/wiki/Relational_database_management_system)). as the `l` in `sql` implies, `sql` is a *language*. it would exist if no database had ever been built.

but databases have been built, and most of them have been built under the same paradigm:

1. tables
2. records
3. links between records (relationships, *not* relations)

*tables* encapsulate single objects or concepts.

for example, a `Person` table

| person_id | person_name |
|-----------|-------------|
| 1         | zach        |

an `Employer` table

| empl_id | empl_name  |
|---------|------------|
| 1       | eri        |
| 2       | georgetown |

and an `EmploymentHistory` table

| empl_hist_id | person_id | empl_id |
|--------------|-----------|---------|
| 1            | 1         | 1       |
| 2            | 1         | 2       |

*records* (rows in those tables) are single instances of that table object

my personal information might exist as a row in the `Person` table, the properites of ERI might exist in the `Employer` table, and my start and (`null`) end date might exist in the `EmploymentHistory` table

*relationships* exist to tie records in one table to records in another

for example, I may have *foreign key* in the `EmploymentHistory` table pointing to records in the `Person` and `Employer` tables

technically speaking, a ***relation*** is the table schema with all the records together (so, a table).

the `sql` query *language* is used to manipulate or query those tables, but the infrastructure which maintains those tables, records, and relationships (*i.e.* manages the relations in that database) is an `rdmbs`.

[here is a ranking of popularity of various `rdmbs` systems](https://db-engines.com/en/ranking/relational+dbms). the most important to know (in order of implementation popularity, with rank in parentheses):

+ `oracle` (1)
+ `mysql` (2)
+ `mssql` (3)
+ `psql` (4)
+ `db2` (6)

**<div align="center">what are your questions so far?</div>**

### a wrinkle: relational `columnar` databases

when we talked about **relations** above, we described them as basically being tables (schema-defined records of information in tabular format). steailng an example from [the column-oriented `dbms` wikipedia page](https://en.wikipedia.org/wiki/Column-oriented_DBMS), assume we have a table `employees`:

| EmpId | Lastname | Firstname | Salary |
|-|-|-|-|
| 10 | Smith | Joe | 40000 |
| 12 | Jones | Mary | 50000 |
| 11 | Johnson | Cathy | 44000 |
| 22 | Jones | Bob | 55000 |

if you imagine what a table looks like in plain text, you're probably thinking of something that looks like what I just showed you -- kind of like a `csv` file. an `rdbms` will create new rows by writing them to some file as a single line that contains one record and all the values for each field in the table for that record:

```
10,Smith,Joe,40000
12,Jones,Mary,50000
11,Johnson,Cathy,44000
22,Jones,Bob,55000
```

that is, you are probably imagining / assuming that the "thing" written in each line of that file is a **row** of data

that `csv`-esque view of our data could also have been transposed and it would still contain the same information:

```
10,12,11,22
Smith,Jones,Johnson,Jones
Joe,Mary,Cathy,Bob
40000,50000,44000,55000
```

the fundamental line item in this file is a **column** of data

for similar numbers of columns and rows this might not make much of a difference, but it makes a huge difference when you have many, many rows. remember that the above look, to a computer, like one long string:

```
A,B,C,D|A,B,C,D|A,B,C,D|A,B,C,D|A,B,C,D|A,B,C,D|A,B,C,D|A,B,C,D|A,B,C,D|A,B,C,D|...
```

and the other

```
A,A,A,A,...|B,B,B,B,...|C,C,C,C,...|D,D,D,D,...|
```

so, who cares?

well, the way that the database chooses to save that information on the disk is actually super important for performance and storage size reasons.

to understand the performance implications, let's pretend we are a `dbms` executing queries.

imagine I asked you to give me all the fields for a person with `empid = 11`. how would you write that query in `sql`?

```sql
select * from employees where empid = 11
```

which of the two file formats would you rather use to do that? remember: you're a computer, you read this as one long line of text

```
10,Smith,Joe,40000|12,Jones,Mary,50000|11,Johnson,Cathy,44000|22,Jones,Bob,55000|
```

or

```
10,12,11,22|Smith,Jones,Johnson,Jones|Joe,Mary,Cathy,Bob|40000,50000,44000,55000|
```

alternatively, suppose I asked you to calculate the total value of the salaries for all of my employees, or to give me a table with *just* the last names. what `sql` code would you write?

```sql
select sum(Salary) as total_salary from employees
select Lastname from employees
```

which of the two file formats would you rather use?

```
10,Smith,Joe,40000|12,Jones,Mary,50000|11,Johnson,Cathy,44000|22,Jones,Bob,55000|
```

or

```
10,12,11,22|Smith,Jones,Johnson,Jones|Joe,Mary,Cathy,Bob|40000,50000,44000,55000|
```

hopefully this pair of examples has helped develop a bit of an intuition for the performance-related use cases of row-based or column-based storage.

the storage-based implications are similar. again imagine you are the database. imagine I ask you to do the following:

1. update Mary Jones' salary
1. insert a new employee with `RowId == 002` and shift all the others forward
1. insert a new employee at the end of the `RowID` queue
1. insert a new column of "middle name" for all records

for each of the above, which file-based arrangement make it easier to write those changes?

one last note: columnar databases allow for special compression options (because each element in the row is of the same data type). this can lead to significant improvements in storage footprint and speed on top of the ones that come naturally from data being laid out in the way our queries prefer

in the end:

+ use **row**-based storage when you are doing **transactional** things: inserting, writing, and updating. having the ability to quickly access all fields of a single record is important
+ use **column**-based storage when you are doing **analytical** things: aggregating all the values in a handfull of columns, or only looking at a small subset of columns

some popular **relational columnar** databases:

+ `sap hana` (19)
+ `vertica` (28)
+ `aws` `redshift` (aka `paraccel`) (32)
+ `google` `bigquery` (33)
+ `greenplum` (38)
+ `kdb+` (financial data) (58)
+ `monetdb` (110)
+ `infinidb` (173)
+ `extremedb` (218)
+ `luciddb` (231)

#### `acid`ity

one last note on database managment and `rdbms`: you will occasionally hear reference to "ACID" or "ACID"ity. this is an acronymn which describes the best-practice principles of database *transactions* which allow multiple different users to share a single representation of data and not break everything.

those properties are:

1. atomicity: every transaction is atomic, or "all or nothing". no partial commits are allowed. if one part fails, it *all* fails (it is "rolled back")
2. consistency: every transaction must take the database from one valid state to another valid state
3. isolation: every transaction should be "isolated" from others until it is committed (information from uncommitted transactions "doesn't exist" outside of that transaction). another way of putting it: the end result of a set of transactions being executed *in parallel* **must** have the same result as if they had been executed *sequentially*
4. durability: once you've committed a transaction it *stays* committed, and can be recovered if needed.

I only bring up `acid` because the above constraints happen to get harder to manage if you want to scale your database to many different servers or parallel processes, or if you want things to be *BLAZING FASTTTTTTTTTT*

**<div align="center">what are your questions so far?</div>**

## the basics: `nosql`

one problem with `rdbms` is that the tables you create must be defined ahead of time -- the object model of the data you are working with is assumed, and if that changes the only way to incorporate that is to update your schema

a second issue is that `acid`ity relies heavily on the existence of one central process (the database server) and one "golden copy" of the underlying data. 

if you want to scale out the number of connected clients or the volume of the data itself, you could keep making your central process bigger and faster ("scale vertically").

at some point, though, you'd like to distribute the workload -- have several servers and access whichever one is free, and create new servers to meet demand

as it turns out, there are several reasons why traditional databases are hard to distribute across multiple tables (*e.g.* joining tables is harder, hard to guarantee consistency or shared state, partitioned or segmented data is a major complication).

> `rdbms` are nice and all, but maybe they aren't worth the hassle. what if we could do something that solved *these* problems (easier to scale, flexible data model) by giving up some of the nice parts (`acid`ity, relationships, or even `sql`)...

and `nosql` is born. `nosql` is a catch-all term for any database system which abandons the **relations** of `rdbms` in favor of some other data model which more directly addresses the above issue. `DROP tables`.

by losing `tables`, we've also necessarily lost `sql`.

really, it might be a little more to-the-point to call it "norel" or "non-relational" but that doesn't sound as `l33t` so I guess I get it.

so we decide to lose relations and `sql`, but we gain a few things.

first, we can tune our performance to be *much, much* faster for some operations (e.g. read, write, and single record lookup) by sacrificing others (aggregation or joins). we can start handling amazon- or google-level data flows.

second -- and this is a big deal -- we can start having a **schemaless** model for our data. we no longer need to be locked in to the fields that we thought we'd have in our model at the get-go. if we suddenly have a new column, we add it! if we don't get a value, we don't store anything (not even a `NULL`).

### extremely hand-wavy run-down of types of `nosql` databases

suppose you've decided that you need to scale your database horizontally and you're willing to accept the cost of breaking the traditional relational data model for database storage and try one of the `nosql` options. which do you choose?

here's a lightning-round, mostly hand-waving description of the structure, motivation, and common implementations for the hottest, l33test `nosql` options out there

#### key-value

you've probably heard references to "key-value pairs" many times before, perhaps in the context of `python` dictionaries. the reason we use dictionaries in `python` (or maps in other languages) is that they are extremely fast for *record insertion* and *lookup*

key-value `nosql` databases are basically just giant distributed dictionaries / hash tables. a *unique* key is associated with a physical place in memory or on disk (via hashing), and the associated value is saved there. the values themselves can be anything: short strings, blobs of text, binary file contents, etc.

key-value stores are **extremely** fast for read, write, update, and delete. at the same time, they are almost completely structureless, so there is effectively no querying possible. you use these when you want to be able to handle extremely large and extremely fast-moving IO reliably and at scale, and not for much else.

the [most popular implementations](https://db-engines.com/en/ranking/key-value+store) are

+ `redis` (7)
+ amazon's `dynamodb` (multimodel) (21)
+ `memcached` (24)
+ `azure` `cosmosdb` (multi-model) (27)
+ `hazelcast` (42)

#### document

document stores are basically key-value stores that have some rules. they still generally associate unique keys to values in the same way, but they make a stipulation that the *values* be "documents" -- text with a format that is generally understood by that database. the commonly accepted structures are things like `json` and `bson` (`b`inary `json`), `xml`, or `yaml`.

by giving up just a bit on the flexibility of key-value stored, document stores are able to allow querying on the database in languages that are generically applicable for the document formats. this is a meaningful improvement if you still have extremely high requirements for throughput but would like to be able to do more than simple lookup queries (e.g. find all `user` documents which have a `location = Washington DC`. 

the [most popular implementations](https://db-engines.com/en/ranking/document+store) are

+ `mongodb` (5)
+ amazon's `dynamodb` (multimodel) (21)
+ `couchbase` (23)
+ `couchdb` (30)
+ `azure` `cosmosdb` (multi-model) (27)
+ `orientdb` (51)
+ google `cloud datastore` (64)

#### "columnar" (AKA wide column store AKA column family databases)

this *awesomely* named family of `nosql` databases is actually **extremely different** from *relational* columnar databases, which leads to basically infinite confusion.

it's so cool! I love it!

the most important thing to take away from this brief rundown is: there are two types of databases people call "columnar" and they are *not the same thing*! this is important because the reasons I would want to use **relational** columnar databases and **`nosql`** columnar databases are generally exact opposites.

that being said: `nosql` columnar databases are *basically* two-dimensional key-value store databases, where the two dimensions of the key are `(row_name, column_name`). sometimes a `timestamp` is thrown in to get to that precious 3D

teh "wide" column store or "column family database" names come about because oftentimes in implementation the columns of the "table" are divvied up into "column families", and different column families are sent to different servers to distribute and scale.

the divvying up of columns is worth some elaboration just to understand the data model. start by think of this as taking a normal table and splitting it into `N` thin and very tall chunks, and distribution those.

for example, assume I have 1M records `row_000000` through `row_999999` and 100 columns `col_00` through `col_99`. I create column families (`col_00 - col_09`, `col_10 - col_19`, etc). I distribute those column families across multiple servers. the first server has records:

```json
{row_000000: {col_00: val_00, ..., col_09: val_09},
 row_000001: {col_00: val_00, ..., col_09: val_09},
 ... ,
 row_999999: {col_00: val_00, ..., col_09: val_09}}
```

when we are asked for a specific `row_name, column_name` pair we can very quickly find go to the server that has the column family chunk with `column_name` in it, then do a fast lookup of `row_name` there. when we get `row_name` we may have to look up `column_name` again to get our value.

like all other key-value stores, these are specifically tuned for *lookup and insertion* -- the **exact opposite** of relational columnar stores!

also, the example above was *dense*: it had a value for ever row and column pair. in practice, columnar `nosql` tables are *sparse*: none of those rows or columns are required, and only real, known values would be written. the example could have read:

```json
{row_000000: {col_07: val_07},
 // no row_000001 record at all! no values for these columns
 ... ,
 row_999999: {col_00: val_00, col_09: val_09}}
```

this is an example of a `nosql` databases being *schemaless*

the [most popular implementations](https://db-engines.com/en/ranking/wide+column+store) are:

+ `cassandra` (11)
+ `hbase` (17)
+ `azure` `cosmosdb` (multi-model) (27)
+ `azure` `table storage` (65)
+ `accumulo` (67)
+ `bigtable` (118)
+ `scylladb` (137)

[this](http://dbmsmusings.blogspot.com/2010/03/distinguishing-two-major-types-of_29.html) is a bit outdated but is one of the better articles I found on the topic, and I borrow a fair amount of the above from the descriptions there

#### graph

much real-world data is not only about entities (records), but *also* about relationships between those entities.

in traditional `rdbms` settings, these are often encoded as "foreign key" constraints. going back to our ugly `EmploymentHistory` table from before:

| empl_hist_id | person_id | empl_id |
|--------------|-----------|---------|
| 1            | 1         | 1       |
| 2            | 1         | 2       |

a record in table `EmploymentHistory` has foreign keys pointing to the `Person` and `Employer` tables indicating there is a relationship between them.

this way of storing relationships is fine for one-hop querying, but is particularly bad for multi-hop querying or relationship pattern matching (see the `neo4j` lecture for more info).

graph databases elevate **relationships** between objects to be of primary importance at the expense of relations (tabular layouts).

most graph databases have two concepts:

+ nodes: generic "things" that have properties. these are often thought of as being nouns in your data's language. properties are arbitrary (it's *schemaless*)
+ edges: relationships between one "thing" and another "thing. these can be directed edges, and they too can have properties. there can be many edges between two nodes.

the same employer example above might be represented internal in a graph database as:

```json
{0: {'edges': {1: ['EMPLOYED_BY'], 2: ['EMPLOYED_BY']},
     'name': 'Zach', 'type': 'Person',
     'properties': {...}},
 1: {'edges': {0: ['EMPLOYES'], 3: [...], ...},
     'name': 'ERI', 'type': 'Employer',
     'properties': {...}},
 2: {'edges': {0: ['EMPLOYES'], 100: [...], ...},
     'name': 'Georgetown', 'type': 'Employer',
     'properties': {...}}}
```

note that you can go from any node (top level key) directly to all it's edges via the `edges` property -- that's the point

the [most popular implementations](https://db-engines.com/en/ranking/graph+dbms) are:

+ `neo4j` (22)
+ `azure` `cosmosdb` (multi-model) (27)
+ `datastax` (43)
+ `orientdb` (51)
+ `arangodb` (63)
+ `virtuoso` (91)
+ `janusgraph` (130)
+ `giraph` (131)
+ amazon `neptune` (135)

### summary / why we talked about this

| database | type | purpose |
|-|-|-|
| `rdbms` (e.g. `postgres`) | relational | structured datastore, fast transactions, no distribution |
| columnar | relational | fast analytical query, warehousing, limited transaction and distribution |
| key-value store | `nosql` | extremely fast throughput, distributed, schemaless, no query |
| document store | `nosql` | extremely fast throughput, distributed, schemaless, limited query |
| wide column store | `nosql` | fast throughput, distributed, schemaless, limited query |
| graph | `nosql` | relational data, relation-based querying (not `sql`) |

given your data (size and structure) and goal, one of the above is probably genuinely a best fit for you. the problem is you usually don't get to choose -- that choice was made by engineers well above you in the data food chain.

given that, it's good to know what the fundamental pieces were in that decision, and what the limitations are of the chosen database.

for ETL there are pros and cons of each type. you are probably more familiar with `sql` as an extraction and computation language, so `nosql` is a hurdle.

for modelling, most of our algorithmic approaches assume tabular data, and relational databases shine. columnar databases fit our purposes well if we are doing aggregation or statistical calculations on subsets of data. when we get down to analytical base tables and proper modelling, we typically anticipate we will be using *all* the columns in that table, and we will be training, validating, and predicting on single records of all features one at a time, and nothing beats a traditioanl `rdbms` for this.

the next few lectures look at `aws` implementations of several of the above types of databases and datastores. we're going to cover the `aws` built-in services:

1. `rds`, the `rdbms` option
    + supports several flavors including `mysql`, `postgrest`, `mssql`, `oracle`, and `mariadb`
1. `dynamodb`, the `nosql` key-value / document store
1. `redshift`, the relational columnar database

in addition we will cover how to use `neo4j` with `ec2` instances

***<div align="center"><code>DROP joke WHERE is_bad</code></div>***
<div align="center"><img src="https://i.redd.it/u0doli6bk2c01.jpg" width="500px"></div>

# END OF LECTURE

next lecture: [`aws rds`](012_dbs_2_rds.ipynb)